### imports

In [13]:
import sys

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard.plugins.hparams import api as hp

import sklearn

sys.path.append('../')
import models


### Generators for loading and augmenting images

In [14]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Setup for experiment

In [15]:
HP_NUM_FREEZED = hp.HParam('num_freezed', hp.Discrete([3, 5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'RMSprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_FREEZED, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Training function

In [16]:
def train_test_model(hparams):
    model = models.XrayModelEfficient(hparams[HP_NUM_FREEZED])
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=hparams[HP_OPTIMIZER],
                  metrics='accuracy')

    model.fit(train_generator,
              steps_per_epoch=100,
              epochs=1)
    
    _, accuracy = model.evaluate(test_generator)
    return accuracy

In [17]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [18]:
session_num = 0

for num_freezed in HP_NUM_FREEZED.domain.values:
  for optimizer in HP_OPTIMIZER.domain.values:
    hparams = {
        HP_NUM_FREEZED: num_freezed,
        HP_OPTIMIZER: optimizer,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'num_freezed': 3, 'optimizer': 'RMSprop'}
41/41 [==============================] - 38s 875ms/step - loss: 1.1125 - accuracy: 0.6638
--- Starting trial: run-1
{'num_freezed': 3, 'optimizer': 'adam'}
41/41 [==============================] - 38s 875ms/step - loss: 0.8847 - accuracy: 0.6638
--- Starting trial: run-2
{'num_freezed': 5, 'optimizer': 'RMSprop'}
41/41 [==============================] - 37s 869ms/step - loss: 1.1086 - accuracy: 0.6638
--- Starting trial: run-3
{'num_freezed': 5, 'optimizer': 'adam'}
41/41 [==============================] - 39s 900ms/step - loss: 0.8349 - accuracy: 0.6638


In [27]:
from collections.abc import Mapping

In [28]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [29]:
%tensorboard --logdir logs/hparam_tuning

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/matined/anaconda3/envs/x-ray/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/tensorboard/default.py", line 33, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 23, in <module>
    from tensorboard import plugin_util
  File "/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 20, in <module>
    from tensorboard._vendor.bleach.sanitizer import Cleaner
  File "/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/tensorboard/_